<a href="https://colab.research.google.com/github/margaretmz/esrgan-e2e-tflite-tutorial/blob/master/ml/add%20metadata/Add%20metadata%20to%20ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Add metadata to ESRGAN
Written by Margaret Maynard-Reid. 
Created on July 21, 2020 | Updated on August 3, 2020 (by Sayak Paul)

This Colab Notebook adds metadata to a tflite model which enables CodeGen in Android Studio with ML Model Binding.

In [1]:
!pip install tflite-support

     |████████████████████████████████| 399kB 3.5MB/s 
  Using cached https://files.pythonhosted.org/packages/89/e3/d576f6f02bc75bacbc3d42494e8f1d063c95617d86648dba243c2cb3963e/pybind11-2.5.0-py2.py3-none-any.whl
  Created wheel for tflite-support: filename=tflite_support-0.1.0a1-cp36-cp36m-linux_x86_64.whl size=3943040 sha256=a0c3428a085790349442e8ae774df6edc1b4cca5cfa216bf24782e25f439b6a9
  Stored in directory: /root/.cache/pip/wheels/5f/65/21/4fa08c598c3e79a1a5260646eb37204d4859fc3da0bfeb8011
Successfully built tflite-support


In [2]:
import os
import tensorflow as tf
from absl import flags

In [3]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

In [4]:
# Create 2 directories, and manually upload the original .tflite to the /model_without_metadata folder
!mkdir model_without_metadata
!mkdir model_with_metadata

In [ ]:
!wget https://github.com/margaretmz/esrgan-e2e-tflite-tutorial/raw/master/ml/add%20metadata/model_without_metadata/esrgan_dr.tflite
!wget https://github.com/margaretmz/esrgan-e2e-tflite-tutorial/raw/master/ml/add%20metadata/model_without_metadata/esrgan_fp16.tflite
!wget https://github.com/margaretmz/esrgan-e2e-tflite-tutorial/raw/master/ml/add%20metadata/model_without_metadata/esrgan_int8.tflite

!mv *.tflite model_without_metadata/

In [14]:
quantization = "int8" #@param ["dr", "fp16", "int8"]
# The original .tflite file without metadata
MODEL_FILE = f"/content/model_without_metadata/esrgan_{quantization}.tflite"
# This is where we will export a new .tflite model file with metadata, and a .json file with metadata info
EXPORT_DIR = "model_with_metadata"

In [15]:
class MetadataPopulatorForGANModel(object):
  """Populates the metadata for the tflite model."""

  def __init__(self, model_file):
    self.model_file = model_file
    self.metadata_buf = None

  def populate(self):
    """Creates metadata and thesn populates it for a style transfer model."""
    self._create_metadata()
    self._populate_metadata()
  
  def _create_metadata(self):
    """Creates the metadata for the tflite model."""

    # Creates model info.
    model_meta = _metadata_fb.ModelMetadataT()
    model_meta
    model_meta.name = "ESRGAN" 
    model_meta.description = ("Enhanced super-res GAN for improving image quality. Converted by TFLiteConverter from TF 2.2.0")
    model_meta.version = "v-2020-07-30"
    model_meta.author = "TensorFlow"
    model_meta.license = ("Apache License. Version 2.0 "
                          "http://www.apache.org/licenses/LICENSE-2.0.")

    # Creates info for the input, original image.
    input_image_meta = _metadata_fb.TensorMetadataT()
    input_image_meta.name = "original_image"
    input_image_meta.description = (
        "The expected image is 128 x 128, with three channels "
        "(red, blue, and green) per pixel. Each value in the tensor is between"
        " 0.0 and 255.0.")
    input_image_meta.content = _metadata_fb.ContentT()
    input_image_meta.content.contentProperties = (
        _metadata_fb.ImagePropertiesT())
    input_image_meta.content.contentProperties.colorSpace = (
        _metadata_fb.ColorSpaceType.RGB)
    input_image_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties)
    input_image_normalization = _metadata_fb.ProcessUnitT()
    input_image_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions)
    input_image_normalization.options = _metadata_fb.NormalizationOptionsT()
    input_image_normalization.options.mean = [0.0]
    input_image_normalization.options.std = [1.0]
    input_image_meta.processUnits = [input_image_normalization]
    input_image_stats = _metadata_fb.StatsT()
    input_image_stats.max = [255]
    input_image_stats.min = [0]
    input_image_meta.stats = input_image_stats

    # Creates output info, anime image
    output_image_meta = _metadata_fb.TensorMetadataT()
    output_image_meta.name = "enhanced_image"
    output_image_meta.description = "Image enhanced."
    output_image_meta.content = _metadata_fb.ContentT()
    output_image_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
    output_image_meta.content.contentProperties.colorSpace = (
        _metadata_fb.ColorSpaceType.RGB)
    output_image_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties)
    output_image_normalization = _metadata_fb.ProcessUnitT()
    output_image_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions)
    output_image_normalization.options = _metadata_fb.NormalizationOptionsT()
    output_image_normalization.options.mean = [0.0]
    output_image_normalization.options.std = [1.0] 
    output_image_meta.processUnits = [output_image_normalization]
    output_image_stats = _metadata_fb.StatsT()
    output_image_stats.max = [255.0]
    output_image_stats.min = [0.0]
    output_image_meta.stats = output_image_stats

    # Creates subgraph info.
    subgraph = _metadata_fb.SubGraphMetadataT()
    subgraph.inputTensorMetadata = [input_image_meta] 
    subgraph.outputTensorMetadata = [output_image_meta] 
    model_meta.subgraphMetadata = [subgraph]

    b = flatbuffers.Builder(0)
    b.Finish(
        model_meta.Pack(b),
        _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
    self.metadata_buf = b.Output()

  def _populate_metadata(self):
    """Populates metadata to the model file."""
    populator = _metadata.MetadataPopulator.with_model_file(self.model_file)
    populator.load_metadata_buffer(self.metadata_buf)
    populator.populate()

In [16]:
def populate_metadata(model_file):
  """Populates the metadata using the populator specified.
  Args:
      model_file: valid path to the model file.
      model_type: a type defined in StyleTransferModelType .
  """

  # Populates metadata for the model.
  model_file_basename = os.path.basename(model_file)
  export_path = os.path.join(EXPORT_DIR, model_file_basename)
  tf.io.gfile.copy(model_file, export_path, overwrite=True)

  populator = MetadataPopulatorForGANModel(export_path) 
  populator.populate()

  # Displays the metadata that was just populated into the tflite model.
  displayer = _metadata.MetadataDisplayer.with_model_file(export_path)
  export_json_file = os.path.join(
      EXPORT_DIR,
      os.path.splitext(model_file_basename)[0] + ".json")
  json_file = displayer.get_metadata_json()
  with open(export_json_file, "w") as f:
    f.write(json_file)
  print("Finished populating metadata and associated file to the model:")
  print(export_path)
  print("The metadata json file has been saved to:")
  print(os.path.join(EXPORT_DIR,
                   os.path.splitext(model_file_basename)[0] + ".json"))

In [17]:
populate_metadata(MODEL_FILE)

Finished populating metadata and associated file to the model:
model_with_metadata/esrgan_int8.tflite
The metadata json file has been saved to:
model_with_metadata/esrgan_int8.json
